In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import inspect

currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
srcdir = parentdir + '/src'
os.sys.path.insert(1, srcdir)

import numpy as np
import copy
import time
import pybullet as p
import pybullet_data
import pinocchio as se3
import matplotlib
import tsid
import numpy.matlib as matlib
from numpy import nan
from numpy.linalg import norm as norm
import commands

import matplotlib.pyplot as plt
from pinbullet_wrapper import PinBulletWrapper
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.utils import zero
from romeo_foot import RomeoFoot

from dotmap import DotMap
import pickle

In [2]:
font = {'family' : 'serif',
        'size'   : 22}

matplotlib.rc('font', **font)
np.set_printoptions(precision=3, linewidth=200, suppress=True)

In [3]:
SCENARIO = 'building'
LOCAL = True
N_EXPS = 100
LOGS = []
TASK = 'balance'

for trial_id in range(N_EXPS):
    file_name = TASK + '_'
    data_path = parentdir + '/data/'
    file_name = file_name + SCENARIO + str(trial_id)
    if LOCAL:
        file_name += '_local'
    data_path += 'ns3/exp_data/'
    exp_data = pickle.load(open( data_path + file_name + '.pkl', "rb" ) )
    if exp_data.success:
        LOGS.append(exp_data.log)
        
import itertools
LOGS_CAT = list(itertools.chain.from_iterable(LOGS))

### Success rate

In [4]:
print len(LOGS)/float(N_EXPS)

1.0


### Performance metrics

In [5]:
print np.mean([100*(np.linalg.norm(data.com.pos - data.com.pos_ref)) for data in LOGS_CAT])
print np.mean([data.slack[1, 0] for data in LOGS_CAT])

1.3869743133694155
0.028853023278008157


### Computation time

In [6]:
print np.mean([1000*data.fullqp_time for data in LOGS_CAT])
print np.max([1000*data.fullqp_time for data in LOGS_CAT])

if LOCAL:
    print np.mean([1000*data.local_time for data in LOGS_CAT])
    print np.max([1000*data.local_time for data in LOGS_CAT])

0.6113356604576111
7.740020751953125
0.12735281705856324
2.026796340942383


### Active set change

In [7]:
M = 100
com_diff = [[] for m in range(M)]
slack = M * [[] for m in range(M)]
K = set()

for data in LOGS_CAT:
    error = DotMap()

    activeset_cached = data.activeset
    activeset_true = data.activeset_fullqp
    activeset_cached = set(np.array(activeset_cached).flatten())
    activeset_true = set(np.array(activeset_true).flatten())
    union = activeset_cached.union(activeset_true)
    intersection = activeset_cached.intersection(activeset_true)
    k = len(union - intersection)
    K.add(k)
    com_diff[k].append(np.linalg.norm(data.com.pos[:,0]-data.com.pos_ref[:,0]))
    slack[k].append(data.slack[1,0])

M = max(K)
for m in range(M):
    print m, np.mean(com_diff[m]), np.mean(slack[m])

0 0.010998504851152155 0.021813600033405777
1 0.04788089294793715 0.18503283603409584
2 0.053953666458022895 0.15187811277744903
3 0.05593852750977434 0.1261086674694019
4 0.05716145455380839 0.12078918727924955
5 0.056024660145259024 0.14007508375234262
6 0.05888341042399997 0.11731616055798012
7 0.058477377519155126 0.12143567608391057
8 0.058184240663854095 0.11625444656650825
9 0.05965647965656773 0.10916369956031388
10 0.060797413301586986 0.10328327755220931
11 0.06481932631486174 0.058311160867565916


### Plots

In [8]:
# N_SIMULATION = exp_data.N_SIMULATION
# dt = exp_data.dt
# N_STEPS = exp_data.N_STEPS
# ONBOARD_WINDOW = exp_data.ONBOARD_WINDOW
# STEP_DURATION = exp_data.STEP_DURATION
# NS3 = exp_data.NS3

# LOG = LOGS[5]
# fig, ax = plt.subplots(1, 3, figsize=(30, 6))
# time = np.arange(0.0, N_SIMULATION*dt, dt)
# axes = ['x', 'y', 'z']
# for i in range(3):
#     err = 100*np.abs([data.com.pos[i,0]-data.com.pos_ref[i,0] for data in LOG])
#     ax[0].plot(time, err, lw=4, label='CoM '+axes[i])
    
# ax[0].grid()
# ax[0].set_xlabel('Time [s]')
# ax[0].set_ylabel('CoM tracking error' + ' [cm]' )
# _ = ax[0].legend(loc='upper left', ncol=1, fancybox=True, shadow=True)
# ax[0].set_ylim([0, 10])

# slack = [data.slack[1, 0] for data in LOG]
# ax[1].plot(time, slack, lw=1)
# ax[1].grid()
# ax[1].set_xlabel('Time [s]')
# ax[1].set_ylabel('Constraint violation' )
# # ax[1].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
# # ax[1].set_ylim([0, 20])

# delays = [data.delay for data in LOG]
# ax[2].grid()
# ax[2].set_xlabel('Time [s]')
# ax[2].set_ylabel('Delay [ms]' )           
# ax[2].plot(time, delays, lw=1, color='k')
# ax[2].set_ylim([0, 180])


# for s in range(N_STEPS + 1):
#     lb = ONBOARD_WINDOW
#     ub = ONBOARD_WINDOW
#     if s == 0:
#         lb = 0
#     if s == N_STEPS:
#         ub = 0
#     ax[2].axvspan(1e-3*(s * STEP_DURATION - lb), 
#                   1e-3*(s * STEP_DURATION + ub), 
#                   alpha=0.5, color='green',
#                   label="_"*s + "Full QP solved onboard")
#     _ = ax[2].legend(loc='upper left', fancybox=True, shadow=True)

# file_name = 'walk_err_'
# if NS3:
#     file_name += SCENARIO
#     file_name += str(IDX)
# if LOCAL:
#     file_name += '_local'
    
# plot_path = parentdir + '/data/plots/'
# # fig.savefig(plot_path + file_name + '.png', bbox_inches='tight')